In [2]:
import pandas as pd
import numpy as np
import pyarrow.dataset as ds
import pvlib

# disable warnings from pandas
pd.options.mode.chained_assignment = None  # default='warn'

# Load data

In [3]:
portugal_metadata = pd.read_excel('data/PortugalPhotovoltaicDataset/pv_plants_metadata.xlsx')
portugal_metadata

,PV Serial Number,Location,Latitude,Longitude,From date,To date,Installed Power (kWp),Connection Power (kWn)
0,84071567,Lisbon,38.728,-9.138,2019-01-01,2022-12-31,46.00,40.0
1,84071569,Lisbon,38.833,-9.191,2019-01-01,2022-12-31,16.32,15.0
2,84071568,Setubal,38.577,-8.872,2019-01-01,2022-12-31,23.52,20.0
3,84071570,Lisbon,38.725,-9.120,2019-01-01,2022-12-31,30.00,27.0
4,84071566,Faro,37.031,-7.893,2019-01-01,2022-12-31,7.00,6.6
5,62030198,Braga,41.493,-8.496,2019-01-01,2022-12-31,64.93,60.0
6,62032213,Lisbon,38.701,-9.236,2019-01-01,2022-12-31,22.54,20.0
7,73060645,Tavira,37.131,-7.645,2019-01-01,2022-12-31,46.00,40.0
8,73061935,Loule,37.131,-8.038,2019-01-01,2022-12-31,46.25,40.0


In [18]:
uk_open_climate_fix_metadata = pd.read_csv("data/UKOpenClimateFix/uk_pv_metadata.csv")
uk_open_climate_fix_metadata.head()

,ss_id,latitude_rounded,longitude_rounded,llsoacd,orientation,tilt,kwp,operational_at
0,2405,53.53,-1.63,E01007430,180.0,35.0,3.36,2010-11-18
1,2406,54.88,-1.38,E01008780,315.0,30.0,1.89,2010-12-03
2,2407,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
3,2408,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03
4,2409,54.88,-1.38,E01008780,225.0,30.0,1.89,2010-12-03


In [4]:
# load portugal dataset
file_path = 'data/PortugalPhotovoltaicDataset/pv_plants_datasets.xlsx'

# define the correspondence table between sheet names and real names
correspondence = {
    'Lisbon_1': ['84071567'],
    'Lisbon_2': ['84071569'],
    'Lisbon_3': ['84071570'],
    'Lisbon_4': ['62032213'], 
    'Setubal': ['84071568'],
    'Faro': ['84071566'],
    'Braga': ['62030198'],
    'Tavira': ['73060645'],
    'Loule': ['73061935']
}

data_frames = []
for real_name, sheet_names in correspondence.items():
    for sheet_name in sheet_names:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        df['serial'] = sheet_name
        df['name'] = real_name
        data_frames.append(df)

portugal_df = pd.concat(data_frames, ignore_index=True)
portugal_df.head()

,Date,Produced Energy (kWh),Specific Energy (kWh/kWp),CO2 Avoided (tons),serial,name
0,2019-01-01 00:00:00,0.0,0.0,,84071567,Lisbon_1
1,2019-01-01 01:00:00,0.0,0.0,,84071567,Lisbon_1
2,2019-01-01 02:00:00,0.0,0.0,,84071567,Lisbon_1
3,2019-01-01 03:00:00,0.0,0.0,,84071567,Lisbon_1
4,2019-01-01 04:00:00,0.0,0.0,,84071567,Lisbon_1


In [12]:
# Open Parquet dataset
dataset = ds.dataset("data/UKOpenClimateFix/uk_pv_30_minute_data.parquet", format="parquet")

# Create a filter for the data
filtered_table = dataset.to_table(filter=(ds.field('ss_id').isin([3062, 3800, 7115, 7497, 8712, 9992, 24865, 24964, 26754, 24962, 19064, 23524, 17446, 10618, 24696])))

# Convert to pandas DataFrame (if needed)
uk_open_climate_fix_df = filtered_table.to_pandas()

# Data preprocessing

## Portugal dataset

In [48]:
def calculate_fixed_azimuth_tilt(latitude, longitude, start_date, end_date):
    results = []

    location = pvlib.location.Location(latitude=latitude, longitude=longitude)
    times = pd.date_range(start=start_date, end=end_date, freq='h')

    clearsky = location.get_clearsky(times)
    solar_position = location.get_solarposition(times)

    tilt_angles = np.arange(0, 90, 5)  # Tilt from 0 to 90 degrees in 5-degree increments
    azimuth_angles = np.arange(0, 180, 10)  # Azimuth from 0 to 180 degrees in 10-degree increments

    # Loop through tilt and azimuth combinations
    for tilt in tilt_angles:
        for azimuth in azimuth_angles:
            # Calculate total irradiance on the panel
            irradiance = pvlib.irradiance.get_total_irradiance(
                surface_tilt=tilt,
                surface_azimuth=azimuth,
                solar_zenith=solar_position['apparent_zenith'],
                solar_azimuth=solar_position['azimuth'],
                dni=clearsky['dni'],
                ghi=clearsky['ghi'],
                dhi=clearsky['dhi'],
                model='isotropic'
            )
            # Sum global irradiance over the year
            total_irradiance = irradiance['poa_global'].sum()
            results.append({'tilt': tilt, 'azimuth': azimuth, 'irradiance': total_irradiance})

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    optimal_row = results_df.loc[results_df['irradiance'].idxmax()]
    optimal_tilt = optimal_row['tilt']
    optimal_azimuth = optimal_row['azimuth']

    return optimal_tilt, optimal_azimuth

## Make data in the same format

In [49]:
# Ensure dates are in datetime format and strip time
portugal_metadata["From date"] = pd.to_datetime(portugal_metadata["From date"]).dt.strftime('%Y-%m-%d')
portugal_metadata["To date"] = pd.to_datetime(portugal_metadata["To date"]).dt.strftime('%Y-%m-%d')

In [50]:
# since in portugal dataset no information about tilt and azimuth, we will calculate the optimal tilt and azimuth and assume that they are fixed

for _, solar_plant in portugal_metadata.iterrows():
    optimal_tilt, optimal_azimuth = calculate_fixed_azimuth_tilt(
        latitude=solar_plant["Latitude"],
        longitude=solar_plant["Longitude"],
        start_date=solar_plant["From date"],
        end_date=solar_plant["To date"]
    )
    portugal_metadata.loc[_, 'optimal_tilt'] = optimal_tilt
    portugal_metadata.loc[_, 'optimal_azimuth'] = optimal_azimuth
    
portugal_metadata

,PV Serial Number,Location,Latitude,Longitude,From date,To date,Installed Power (kWp),Connection Power (kWn),optimal_tilt,optimal_azimuth
0,84071567,Lisbon,38.728,-9.138,2019-01-01,2022-12-31,46.00,40.0,35.0,170.0
1,84071569,Lisbon,38.833,-9.191,2019-01-01,2022-12-31,16.32,15.0,35.0,170.0
2,84071568,Setubal,38.577,-8.872,2019-01-01,2022-12-31,23.52,20.0,35.0,170.0
3,84071570,Lisbon,38.725,-9.120,2019-01-01,2022-12-31,30.00,27.0,35.0,170.0
4,84071566,Faro,37.031,-7.893,2019-01-01,2022-12-31,7.00,6.6,35.0,170.0
5,62030198,Braga,41.493,-8.496,2019-01-01,2022-12-31,64.93,60.0,40.0,170.0
6,62032213,Lisbon,38.701,-9.236,2019-01-01,2022-12-31,22.54,20.0,35.0,170.0
7,73060645,Tavira,37.131,-7.645,2019-01-01,2022-12-31,46.00,40.0,35.0,170.0
8,73061935,Loule,37.131,-8.038,2019-01-01,2022-12-31,46.25,40.0,35.0,170.0


In [51]:
portugal_df = portugal_df[["serial", "name", "Date", "Produced Energy (kWh)"]]
portugal_df.rename(
    columns={
        "Date": "datetime",
        "Produced Energy (kWh)": "produced energy"
    },
    inplace=True
)
portugal_df.head()

,serial,name,datetime,produced energy
0,84071567,Lisbon_1,2019-01-01 00:00:00,0.0
1,84071567,Lisbon_1,2019-01-01 01:00:00,0.0
2,84071567,Lisbon_1,2019-01-01 02:00:00,0.0
3,84071567,Lisbon_1,2019-01-01 03:00:00,0.0
4,84071567,Lisbon_1,2019-01-01 04:00:00,0.0


In [52]:
# Merge portugal_df with portugal_metadata based on the "serial" column
portugal_df['serial'] = portugal_df['serial'].astype(str)
portugal_metadata['PV Serial Number'] = portugal_metadata['PV Serial Number'].astype(str)

portugal_df = portugal_df.merge(
    portugal_metadata[["PV Serial Number", "Installed Power (kWp)"]],  # Columns to bring
    left_on="serial",  # Column in portugal_df
    right_on="PV Serial Number",  # Column in portugal_metadata
    how="left"  # Type of join
)

portugal_df.rename(
    columns={
        "Installed Power (kWp)": "kwp"
    },
    inplace=True
)

portugal_df['produced energy'] = portugal_df['produced energy'].round(2)  # Round to 2 decimal places
portugal_df.drop(columns=["PV Serial Number"], inplace=True)

In [53]:
# Extract date and time features
portugal_df['year'] = portugal_df['datetime'].dt.year
portugal_df['month'] = portugal_df['datetime'].dt.month
portugal_df['day'] = portugal_df['datetime'].dt.day
portugal_df['hour'] = portugal_df['datetime'].dt.hour
portugal_df['minute'] = portugal_df['datetime'].dt.minute

portugal_df.drop(columns=["datetime"], inplace=True)


# UK dataset

In [13]:
# Check missing values
print("Missing values before cleanup:")
print(uk_open_climate_fix_df.isna().sum())

# Drop NaN values and reset index
uk_open_climate_fix_df = uk_open_climate_fix_df.dropna().reset_index(drop=True)

print("\nMissing values after cleanup:")
print(uk_open_climate_fix_df.isna().sum())

Missing values before cleanup:
generation_wh    4
datetime         0
ss_id            0
dtype: int64

Missing values after cleanup:
generation_wh    0
datetime         0
ss_id            0
dtype: int64


In [6]:
def group_complete_hours_by_plant(df, datetime_col, energy_col, plant_id_col, plant_id):
    """
    Groups data by complete hours for a specific PV plant where both 30-minute intervals are present.

    Parameters:
    df : pd.DataFrame - The input DataFrame.
    datetime_col : str - The name of the datetime column.
    energy_col : str - The name of the energy production column.
    plant_id_col : str - The column name for the power plant ID.
    plant_id : str or int - The ID of the specific power plant to process.

    Returns:
    pd.DataFrame - Data grouped by hour with only complete records for the selected plant.
    """
    # Filter the data for the specific plant
    plant_data = df[df[plant_id_col] == plant_id].copy()

    initial_size = plant_data.shape

    # Ensure datetime column is in datetime format
    plant_data[datetime_col] = pd.to_datetime(plant_data[datetime_col])

    # Extract hour and minute from the timestamp for grouping
    plant_data['hour'] = plant_data[datetime_col].dt.floor('h')
    plant_data['minute'] = plant_data[datetime_col].dt.minute

    # Count the number of records for each hour
    hour_counts = plant_data.groupby('hour')[datetime_col].count().reset_index(name='count')

    # Identify incomplete hours (count != 2)
    incomplete_hours = hour_counts[hour_counts['count'] != 2]
    num_incomplete_pairs = incomplete_hours.shape[0]

    # Keep only hours with exactly 2 records (both 30-minute intervals)
    complete_hours = hour_counts[hour_counts['count'] == 2]['hour']

    # Filter the plant data to only include complete hours
    complete_plant_data = plant_data[plant_data['hour'].isin(complete_hours)].copy()

    # Group by hour and aggregate energy production
    grouped_df = complete_plant_data.groupby('hour')[energy_col].sum().reset_index()
    grouped_df.rename(columns={'hour': datetime_col}, inplace=True)

    print(f"Initial size: {initial_size}")
    print(f"Grouped size: {grouped_df.shape}")
    print(f"Number of incomplete hour pairs (missing :00 or :30): {num_incomplete_pairs}")

    return grouped_df

In [14]:
uk_open_climate_fix_df_temp = []

for ss_id in set(uk_open_climate_fix_df["ss_id"]):
    pv_grouped = group_complete_hours_by_plant(uk_open_climate_fix_df, "datetime", "generation_wh", "ss_id", ss_id)
    pv_grouped["ss_id"] = ss_id
    uk_open_climate_fix_df_temp.append(pv_grouped)
    
uk_open_climate_fix_df_selected_hourly = pd.concat(uk_open_climate_fix_df_temp)

Initial size: (66000, 3)
Grouped size: (33000, 2)
Number of incomplete hour pairs (missing :00 or :30): 0
Initial size: (72672, 3)
Grouped size: (36336, 2)
Number of incomplete hour pairs (missing :00 or :30): 0
Initial size: (81168, 3)
Grouped size: (40584, 2)
Number of incomplete hour pairs (missing :00 or :30): 0
Initial size: (74640, 3)
Grouped size: (37320, 2)
Number of incomplete hour pairs (missing :00 or :30): 0
Initial size: (75168, 3)
Grouped size: (37584, 2)
Number of incomplete hour pairs (missing :00 or :30): 0
Initial size: (77760, 3)
Grouped size: (38880, 2)
Number of incomplete hour pairs (missing :00 or :30): 0
Initial size: (128732, 3)
Grouped size: (64366, 2)
Number of incomplete hour pairs (missing :00 or :30): 0
Initial size: (134112, 3)
Grouped size: (67056, 2)
Number of incomplete hour pairs (missing :00 or :30): 0
Initial size: (128880, 3)
Grouped size: (64440, 2)
Number of incomplete hour pairs (missing :00 or :30): 0
Initial size: (139392, 3)
Grouped size: (69

## Format to the same as portugal dataset
Convert Wh to kWh

In [19]:
uk_open_climate_fix_df_selected_hourly['produced energy'] = uk_open_climate_fix_df_selected_hourly['generation_wh'] / 1000 # Convert Wh to kWh
uk_open_climate_fix_df_selected_hourly['produced energy'] = uk_open_climate_fix_df_selected_hourly['produced energy'].round(2) # Round to 2 decimal places
uk_open_climate_fix_df_selected_hourly.drop(columns=["generation_wh"], inplace=True)

uk_open_climate_fix_df_selected_hourly['ss_id'] = uk_open_climate_fix_df_selected_hourly['ss_id'].astype(str)
uk_open_climate_fix_metadata['ss_id'] = uk_open_climate_fix_metadata['ss_id'].astype(str)

uk_open_climate_fix_df_selected_hourly = uk_open_climate_fix_df_selected_hourly.merge(
    uk_open_climate_fix_metadata[["ss_id", "kwp"]],  # Columns to bring
    left_on="ss_id",  # Column in portugal_df
    right_on="ss_id",  # Column in portugal_metadata
    how="left"  # Type of join
)

# align the column names with the portugal dataset
uk_open_climate_fix_df_selected_hourly.rename(
    columns={
        "ss_id": "serial"
    },
    inplace=True
)

KeyError: 'generation_wh'

In [16]:
# Extract date and time features
uk_open_climate_fix_df_selected_hourly['year'] = uk_open_climate_fix_df_selected_hourly['datetime'].dt.year
uk_open_climate_fix_df_selected_hourly['month'] = uk_open_climate_fix_df_selected_hourly['datetime'].dt.month
uk_open_climate_fix_df_selected_hourly['day'] = uk_open_climate_fix_df_selected_hourly['datetime'].dt.day
uk_open_climate_fix_df_selected_hourly['hour'] = uk_open_climate_fix_df_selected_hourly['datetime'].dt.hour
uk_open_climate_fix_df_selected_hourly['minute'] = uk_open_climate_fix_df_selected_hourly['datetime'].dt.minute

uk_open_climate_fix_df_selected_hourly.drop(columns=["datetime"], inplace=True)

# Save datasets

In [59]:
portugal_metadata.to_csv("data/PortugalPhotovoltaicDataset/photovoltaic_dataset_metadata.csv", index=False)

In [10]:
uk_open_climate_fix_metadata.to_csv("data/UKOpenClimateFix/uk_open_climate_fixed_metadata.csv", index=False)

In [60]:
portugal_df.to_csv("data/PortugalPhotovoltaicDataset/photovoltaic_dataset_raw.csv", index=False)

In [17]:
uk_open_climate_fix_df_selected_hourly.to_csv("data/UKOpenClimateFix/uk_open_climate_hourly_dataset_raw.csv", index=False)